In [ ]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [ ]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [ ]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'ASIANPAINT',
       'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP',
       'DMART', 'FINEORG', 'GICRE', 'GILLETTE', 'GLAND', 'HDFCAMC',
       'ICICIGI', 'INDIGOPNTS', 'INDUSINDBK', 'INFY', 'IRCTC',
       'JMFINANCIL', 'JUBLFOOD', 'LUXIND', 'NIACL', 'PAGEIND',
       'PIDILITIND', 'PNB', 'RELAXO', 'RELIANCE', 'TEAMLEASE',
       'UJJIVANSFB', 'VAIBHAVGBL', 'VALIANTORG', 'WIPRO'], dtype=object)

In [ ]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    stock_df['Stock'] = stock_symbol
    
    stock_df['Close'] = round(stock_df['Close'],0)     
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]   
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]   
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]   
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [ ]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [ ]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%'] 

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
previous = round(sum(tmp_df['Prev_Close']*tmp_df['Shares']),0)

today_pnl_amount = current-previous
today_pnl_percentage = round((current-previous)*100/previous,2)

curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df['Risk_Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

print('Investment: ', investment)
print('Current: ', current)
print('Today PnL: ', today_pnl_amount)
print('Today PnL %: ', today_pnl_percentage)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 35
Investment:  1666503.0
Current:  1720711.0
Today PnL:  14797.0
Today PnL %:  0.87
Current PnL:  54208.0
Current PnL %:  3.25
Estimate:  3035295.0
Estimate PnL:  1368792.0
Estimate PnL %:  82.14


In [ ]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk_Ind
33,621.0,VALIANTORG.NS,1851.0,595.0,481.73,64.0,2T3Y,2T3Y,30830.72,39744.0,38080.0,118464.0,28.91,284.24,255.33,1.85,53.0
0,368.0,5PAISA.NS,553.0,356.0,311.17,97.0,V40N,ATH,30183.49,35696.0,34532.0,53641.0,18.26,77.72,59.46,1.81,33.0
16,1225.0,ICICIGI.NS,1620.0,1182.0,1106.17,46.0,V40,ATH,50883.82,56350.0,54372.0,74520.0,10.74,46.45,35.71,3.05,33.0
6,7040.0,BAJFINANCE.NS,7901.0,6990.0,6365.23,16.0,V40,ATH,101843.68,112640.0,111840.0,126416.0,10.60,24.13,13.53,6.11,65.0
7,268.0,BANDHANBNK.NS,723.0,268.0,242.80,125.0,2T3Y,2T3Y,30350.00,33500.0,33500.0,90375.0,10.38,197.78,187.40,1.82,19.0
26,2652.0,PIDILITIND.NS,2902.0,2608.0,2442.09,20.0,V40,ATH,48841.80,53040.0,52160.0,58040.0,8.60,18.83,10.23,2.93,25.0
12,186.0,GICRE.NS,358.0,184.0,171.85,175.0,2T3Y,2T3Y,30073.75,32550.0,32200.0,62650.0,8.23,108.32,100.09,1.80,15.0
31,36.0,UJJIVANSFB.NS,57.0,37.0,33.60,900.0,V40N,ATH,30240.00,32400.0,33300.0,51300.0,7.14,69.64,62.50,1.81,13.0
4,3241.0,ASIANPAINT.NS,3566.0,3193.0,3048.30,17.0,V40,ATH,51821.10,55097.0,54281.0,60622.0,6.32,16.98,10.66,3.11,20.0
34,407.0,WIPRO.NS,713.0,404.0,382.90,130.0,2T3Y,2T3Y,49777.00,52910.0,52520.0,92690.0,6.29,86.21,79.92,2.99,19.0


In [ ]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk_Ind
33,621.0,VALIANTORG.NS,1851.0,595.0,481.73,64.0,2T3Y,2T3Y,30830.72,39744.0,38080.0,118464.0,28.91,284.24,255.33,1.85,53.0
14,932.0,GLAND.NS,4316.0,924.0,1354.78,37.0,2T3Y,2T3Y,50126.86,34484.0,34188.0,159692.0,-31.21,218.58,249.79,3.01,-94.0
23,1506.0,LUXIND.NS,4546.0,1482.0,1499.87,34.0,V40N,2T3Y,50995.58,51204.0,50388.0,154564.0,0.41,203.09,202.68,3.06,1.0
32,301.0,VAIBHAVGBL.NS,966.0,301.0,320.64,96.0,2T3Y,2T3Y,30781.44,28896.0,28896.0,92736.0,-6.13,201.27,207.40,1.85,-11.0
7,268.0,BANDHANBNK.NS,723.0,268.0,242.80,125.0,2T3Y,2T3Y,30350.00,33500.0,33500.0,90375.0,10.38,197.78,187.40,1.82,19.0
24,120.0,NIACL.NS,328.0,119.0,116.60,260.0,TBD,ATH,30316.00,31200.0,30940.0,85280.0,2.92,181.30,178.38,1.82,5.0
30,2249.0,TEAMLEASE.NS,5281.0,2222.0,2239.86,14.0,V40N,2T3Y,31358.04,31486.0,31108.0,73934.0,0.41,135.77,135.36,1.88,1.0
1,1391.0,AAVAS.NS,3196.0,1398.0,1405.00,15.0,2T3Y,2T3Y,21075.00,20865.0,20970.0,47940.0,-1.00,127.47,128.47,1.26,-1.0
17,1473.0,INDIGOPNTS.NS,3112.0,1456.0,1408.25,83.0,V40N,2T3Y,116884.75,122259.0,120848.0,258296.0,4.60,120.98,116.38,7.01,32.0
12,186.0,GICRE.NS,358.0,184.0,171.85,175.0,2T3Y,2T3Y,30073.75,32550.0,32200.0,62650.0,8.23,108.32,100.09,1.80,15.0


In [ ]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk_Ind
26,2652.0,PIDILITIND.NS,2902.0,2608.0,2442.09,20.0,V40,ATH,48841.80,53040.0,52160.0,58040.0,8.60,18.83,10.23,2.93,25.0
4,3241.0,ASIANPAINT.NS,3566.0,3193.0,3048.30,17.0,V40,ATH,51821.10,55097.0,54281.0,60622.0,6.32,16.98,10.66,3.11,20.0
6,7040.0,BAJFINANCE.NS,7901.0,6990.0,6365.23,16.0,V40,ATH,101843.68,112640.0,111840.0,126416.0,10.60,24.13,13.53,6.11,65.0
29,2463.0,RELIANCE.NS,2811.0,2470.0,2464.49,21.0,V40,ATH,51754.29,51723.0,51870.0,59031.0,-0.06,14.06,14.12,3.11,-0.0
5,1450.0,BAJAJFINSV.NS,1907.0,1451.0,1449.21,35.0,V40,NaN,50722.35,50750.0,50785.0,66745.0,0.05,31.59,31.54,3.04,0.0
16,1225.0,ICICIGI.NS,1620.0,1182.0,1106.17,46.0,V40,ATH,50883.82,56350.0,54372.0,74520.0,10.74,46.45,35.71,3.05,33.0
25,39133.0,PAGEIND.NS,53631.0,38776.0,39840.50,2.0,V40,ATH,79681.00,78266.0,77552.0,107262.0,-1.78,34.61,36.39,4.78,-9.0
3,429.0,AMBUJACEM.NS,588.0,424.0,408.85,120.0,TBD,ATH,49062.00,51480.0,50880.0,70560.0,4.93,43.82,38.89,2.94,14.0
8,1578.0,BATAINDIA.NS,2173.0,1584.0,1501.51,67.0,V40,ATH,100601.17,105726.0,106128.0,145591.0,5.09,44.72,39.63,6.04,31.0
9,4294.0,BAYERCROP.NS,6002.0,4229.0,4125.00,7.0,V40N,NaN,28875.00,30058.0,29603.0,42014.0,4.10,45.50,41.40,1.73,7.0


In [ ]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk_Ind
17,1473.0,INDIGOPNTS.NS,3112.0,1456.0,1408.25,83.0,V40N,2T3Y,116884.75,122259.0,120848.0,258296.0,4.60,120.98,116.38,7.01,32.0
6,7040.0,BAJFINANCE.NS,7901.0,6990.0,6365.23,16.0,V40,ATH,101843.68,112640.0,111840.0,126416.0,10.60,24.13,13.53,6.11,65.0
8,1578.0,BATAINDIA.NS,2173.0,1584.0,1501.51,67.0,V40,ATH,100601.17,105726.0,106128.0,145591.0,5.09,44.72,39.63,6.04,31.0
25,39133.0,PAGEIND.NS,53631.0,38776.0,39840.50,2.0,V40,ATH,79681.00,78266.0,77552.0,107262.0,-1.78,34.61,36.39,4.78,-9.0
13,4428.0,GILLETTE.NS,7529.0,4383.0,4479.47,14.0,V40,ATH,62712.58,61992.0,61362.0,105406.0,-1.15,68.08,69.23,3.76,-4.0
4,3241.0,ASIANPAINT.NS,3566.0,3193.0,3048.30,17.0,V40,ATH,51821.10,55097.0,54281.0,60622.0,6.32,16.98,10.66,3.11,20.0
29,2463.0,RELIANCE.NS,2811.0,2470.0,2464.49,21.0,V40,ATH,51754.29,51723.0,51870.0,59031.0,-0.06,14.06,14.12,3.11,-0.0
10,3501.0,DMART.NS,5324.0,3471.0,3702.36,14.0,SS,ATH,51833.04,49014.0,48594.0,74536.0,-5.44,43.80,49.24,3.11,-17.0
23,1506.0,LUXIND.NS,4546.0,1482.0,1499.87,34.0,V40N,2T3Y,50995.58,51204.0,50388.0,154564.0,0.41,203.09,202.68,3.06,1.0
22,492.0,JUBLFOOD.NS,903.0,483.0,468.21,109.0,SS,ATH,51034.89,53628.0,52647.0,98427.0,5.08,92.86,87.78,3.06,16.0


In [ ]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/Stock-Portfolio-Analysis.csv', index=False)

In [ ]:
tmp_df.sort_values(by = 'Risk_Ind', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk_Ind
14,932.0,GLAND.NS,4316.0,924.0,1354.78,37.0,2T3Y,2T3Y,50126.86,34484.0,34188.0,159692.0,-31.21,218.58,249.79,3.01,-94.0
10,3501.0,DMART.NS,5324.0,3471.0,3702.36,14.0,SS,ATH,51833.04,49014.0,48594.0,74536.0,-5.44,43.80,49.24,3.11,-17.0
32,301.0,VAIBHAVGBL.NS,966.0,301.0,320.64,96.0,2T3Y,2T3Y,30781.44,28896.0,28896.0,92736.0,-6.13,201.27,207.40,1.85,-11.0
25,39133.0,PAGEIND.NS,53631.0,38776.0,39840.50,2.0,V40,ATH,79681.00,78266.0,77552.0,107262.0,-1.78,34.61,36.39,4.78,-9.0
13,4428.0,GILLETTE.NS,7529.0,4383.0,4479.47,14.0,V40,ATH,62712.58,61992.0,61362.0,105406.0,-1.15,68.08,69.23,3.76,-4.0
2,621.0,AMARAJABAT.NS,988.0,601.0,626.83,48.0,V200,ATH,30087.84,29808.0,28848.0,47424.0,-0.93,57.62,58.55,1.81,-2.0
1,1391.0,AAVAS.NS,3196.0,1398.0,1405.00,15.0,2T3Y,2T3Y,21075.00,20865.0,20970.0,47940.0,-1.00,127.47,128.47,1.26,-1.0
18,1290.0,INDUSINDBK.NS,1987.0,1287.0,1288.70,24.0,SS,ATH,30928.80,30960.0,30888.0,47688.0,0.10,54.19,54.09,1.86,0.0
29,2463.0,RELIANCE.NS,2811.0,2470.0,2464.49,21.0,V40,ATH,51754.29,51723.0,51870.0,59031.0,-0.06,14.06,14.12,3.11,-0.0
5,1450.0,BAJAJFINSV.NS,1907.0,1451.0,1449.21,35.0,V40,NaN,50722.35,50750.0,50785.0,66745.0,0.05,31.59,31.54,3.04,0.0
